In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

%matplotlib inline

In [97]:
housing = pd.read_csv('datasets/train.csv') #sat_2017 =pd.read_csv("data/sat_2017.csv")

In [98]:
housing.shape

(2051, 81)

In [80]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [5]:
housing.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [6]:
#Determine _what_ missing values mean.

In [7]:
housing.loc[:,("Alley","Pool QC")].head()

,Alley,Pool QC
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [8]:
#Pool QC 

In [9]:
#Misc Feature

In [10]:
#housing["Fence"].unique()

In [11]:
#housing["Alley"].unique()

In [12]:
#housing["Fireplace Qu"].unique()

In [13]:
#housing["Lot Frontage"].unique()

In [14]:
#housing["Misc Feature"].unique()

In [15]:
total = housing.isnull().sum().sort_values(ascending=False)
percent = (housing.isnull().sum()/housing.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
Pool QC,2042,0.995612
Misc Feature,1986,0.968308
Alley,1911,0.931741
Fence,1651,0.804973
Fireplace Qu,1000,0.487567
Lot Frontage,330,0.160897
Garage Finish,114,0.055583
Garage Cond,114,0.055583
Garage Qual,114,0.055583
Garage Yr Blt,114,0.055583


In [16]:
#housing = housing.drop((missing_data[missing_data['Percent'] <= 0.05]))

In [17]:
housing.groupby(["SalePrice","Neighborhood"]).describe

<bound method GroupBy.describe of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000022BD62D1E88>>

In [18]:
housing["Pool QC"].unique()

array([nan, 'Fa', 'Gd', 'Ex', 'TA'], dtype=object)

In [19]:
#housing.Pool_QC.groupby(('SalePrice, as_index=False).mean())

SyntaxError: EOL while scanning string literal (<ipython-input-19-f477503e041e>, line 1)

In [ ]:
#housing.groupby(["SalePrice","Pool QC"], as_index=False).mean()

In [81]:
N = housing.loc[:,("SalePrice","Neighborhood")]

In [82]:
N.head()

,SalePrice,Neighborhood
0,130500,Sawyer
1,220000,SawyerW
2,109000,NAmes
3,174000,Timber
4,138500,SawyerW


In [83]:
N["Neighborhood"].unique()


array(['Sawyer', 'SawyerW', 'NAmes', 'Timber', 'Edwards', 'OldTown',
       'BrDale', 'CollgCr', 'Somerst', 'Mitchel', 'StoneBr', 'NridgHt',
       'Gilbert', 'Crawfor', 'IDOTRR', 'NWAmes', 'Veenker', 'MeadowV',
       'SWISU', 'NoRidge', 'ClearCr', 'Blmngtn', 'BrkSide', 'NPkVill',
       'Blueste', 'GrnHill', 'Greens', 'Landmrk'], dtype=object)

In [84]:
N["Neighborhood"].mean

<bound method Series.mean of 0        Sawyer
1       SawyerW
2         NAmes
3        Timber
4       SawyerW
         ...   
2046     Timber
2047    Edwards
2048    Crawfor
2049      NAmes
2050    Gilbert
Name: Neighborhood, Length: 2051, dtype: object>

In [85]:
Rank = N.groupby(["Neighborhood"], as_index=False).median().sort_values(by=["SalePrice"])

In [86]:
Rank_mapper ={"MeadowV": 1,"IDOTRR":2  ,"BrDale":3 ,"OldTown":4,"BrkSide":5,"Edwards":6,
              "SWISU":7,"Landmrk":8,"Sawyer":9,"NPkVill":10,"Blueste":11,"NAmes":12,"Mitchel":13,
              "SawyerW":14,"Greens":15,"Gilbert":16,"NWAmes":17,"Blmngtn":18,"CollgCr":19,
              "Crawfor":20,"ClearCr":21,"Somerst":22,"Timber":23,"Veenker":24,"GrnHill":25,
              "NoRidge":26,"NridgHt":27,"StoneBr":28}

In [87]:
N["Neighborhood"] = N["Neighborhood"].replace(Rank_mapper)

In [35]:
N

,SalePrice,Neighborhood
0,130500,9
1,220000,14
2,109000,12
3,174000,23
4,138500,14
...,...,...
2046,298751,23
2047,82500,6
2048,177000,20
2049,144000,12


In [ ]:
#Housing.drop(""Neighborhood"").append("N")

In [88]:
N.rename(columns={'Neighborhood':'Neighborhood_e',
                          'SalePrice':'SalePrice_e'}, 
                 inplace=True)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79.0,11449,Pave,NaN,IR1,HLS,...,0,0,NaN,NaN,NaN,0,1,2008,WD,298751
2047,785,905377130,30,RL,NaN,12342,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2009,WD,82500
2048,916,909253010,50,RL,57.0,7558,Pave,NaN,Reg,Bnk,...,0,0,NaN,NaN,NaN,0,3,2009,WD,177000
2049,639,535179160,20,RL,80.0,10400,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2009,WD,144000


In [ ]:
#pd.concat([df1, df4], axis=1, join='inner')

In [89]:
housing =housing.append(N,ignore_index=True, sort=False) 

In [43]:
#housing.drop(["Neighborhood"], inplace = True, axis=1)

In [49]:
housing.shape

(4102, 82)

In [90]:
housing.drop(["SalePrice_e"], inplace = True, axis=1)

In [91]:
housing.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

In [53]:
plt.["SalePrice","Neighborhood_e"]

SyntaxError: invalid syntax (<ipython-input-53-6481783474c3>, line 1)

In [95]:
#sns.scatterplot(data =housing, x = "Neighborhood_e", y="SalePrice")  sns.scatterplot(x="total_bill", y="tip", data=tips)
#drinks.beer.plot(kind='density', xlim=(0, 500));
sns.scatterplot(data = N, x = "Neighborhood_e")

ValueError: Either both or neither of `x` and `y` must be specified (but try passing to `data`, which is more flexible).